In [1]:
!pip install streamlit -q

     |████████████████████████████████| 9.7 MB 5.2 MB/s 
     |████████████████████████████████| 4.3 MB 36.4 MB/s 
     |████████████████████████████████| 180 kB 48.4 MB/s 
     |████████████████████████████████| 76 kB 4.9 MB/s 
     |████████████████████████████████| 111 kB 52.4 MB/s 
     |████████████████████████████████| 164 kB 48.0 MB/s 
     |████████████████████████████████| 63 kB 1.7 MB/s 
     |████████████████████████████████| 128 kB 48.0 MB/s 
     |████████████████████████████████| 792 kB 46.2 MB/s 
     |████████████████████████████████| 380 kB 48.6 MB/s 
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
jupyter-console 5.2.0 requires prompt-toolkit<2.0.0,>=1.0.0, but you have prompt-toolkit 3.0.28 which is incompatible.
google-colab 1.0.0 requires ipykernel~=4.10, but you have ipykernel 6.9.1 which is incompatible.
google-colab 1.0.0 requires ipytho

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!pip uninstall tensorflow
!pip install tensorflow==2.4.2

In [ ]:
%%writefile GmailSmartCompose.py

import streamlit as st
import pickle
import tensorflow as tf
from tensorflow.keras.models import load_model
import re
from numpy import zeros, argmax
from tensorflow.keras.preprocessing.sequence import pad_sequences
from IPython.display import Javascript
from IPython.display import display


header_text = '<p style="font-family: Garamond, serif; color:red; font-weight: bold; font-size: 80px;">Welcome to Gmail</p>'
st.markdown(header_text, unsafe_allow_html=True)

st.write('')
st.write('')

subheader_text = '<p style="font-family: Garamond, serif; color:DodgerBlue; font-weight: bold; font-size: 25px;">Compose your Mail</p>'
st.markdown(subheader_text, unsafe_allow_html=True)

info = '<p style="font-family: Garamond, serif; color:Green; font-weight: bold; font-size: 40px;">Info</p>'
st.sidebar.markdown(info, unsafe_allow_html=True)

# Blog
if st.sidebar.button('Go to Blog'):
  url = 'https://medium.com/@mallugarisantoshreddy/gmail-smart-compose-a-smart-way-of-writing-an-email-6727cad95ac8#edcb'
  display(Javascript('window.open("{url}");'.format(url=url)))


# Resume
st.sidebar.write('')
st.sidebar.write('')
with open(r"Mallugari_Santosh_Reddy.pdf","rb") as pdf_file:
      pdf_byte = pdf_file.read()
      
st.sidebar.download_button(label="Download Resume", data=pdf_byte, file_name="Resume.pdf", mime='application/octet-stream')



# contact Information
st.sidebar.write('')
st.sidebar.write('')
st.sidebar.write('')

contact = st.sidebar.selectbox('How would you like to contact me?',('None','Email','Phone'))

if contact == 'Email':
    st.write('')
    mail = '<p style="font-family: Garamond, serif; color:orange; font-weight: bold; font-size: 20px;">  Mallugarisantoshreddy@gmail.com</p>'
    st.sidebar.markdown(mail, unsafe_allow_html=True)
    
elif contact == 'Phone':
    phone = '<p style="font-family: Garamond, serif; color:orange; font-weight: bold; font-size: 25px;">  +91 8106057568</p>'
    st.sidebar.markdown(phone, unsafe_allow_html=True)
    
# Social Information
st.sidebar.write('')
social = st.sidebar.selectbox('Social Links',('None','LinkedIn', 'Github'))

if social == 'LinkedIn':
  url = "https://www.linkedin.com/in/mallugari-santosh-reddy-8315121a1/"
  display(Javascript('window.open("{url}");'.format(url=url)))
    
if social == 'Github':
  url = 'https://github.com/santu3217'
  display(Javascript('window.open("{url}");'.format(url=url)))
    
    
# projects
st.sidebar.write('')
projects = st.sidebar.selectbox('Other Projects',('None','Hindi Character Recognition'))

project_button = None
project_model = None

if projects == 'Hindi Character Recognition':
    project_button = st.sidebar.button('Blog')
    project_model = st.sidebar.button('Model')
        
if project_button:
  url = 'https://medium.com/@mallugarisantoshreddy/hindi-character-recognition-4f506a8b4d82'
  display(Javascript('window.open("{url}");'.format(url=url)))
    
    
if project_model:
  url = 'https://www.linkedin.com/in/mallugari-santosh-reddy-8315121a1/'
  display(Javascript('window.open("{url}");'.format(url=url)))
    


##########################################################################################################################

def perplexity(y_true, y_pred):
    return tf.keras.backend.exp(tf.keras.backend.mean(tf.keras.backend.sparse_categorical_crossentropy(y_true, y_pred)))
 
test_decoder_model = load_model('Test_Decoder_Model.h5')
test_encoder_model = load_model('Test_Encoder_Model.h5')
    
file = open('GmailSmartCompose_Tokenizer.pickle','rb')
Tokenizer = pickle.load(file)

max_length_in = 21    
max_length_out = 19

index_to_word = dict(map(reversed, Tokenizer.word_index.items()))

def tokenize_text(text):
  text = '<start> ' + text.lower() + ' <end>'
  text_tensor = Tokenizer.texts_to_sequences([text])
  text_tensor = pad_sequences(text_tensor, maxlen=max_length_in, padding="post")
  return text_tensor

def decode_sequence_attention(input_tensor):
    # Encode the input as state vectors.
    enc_out, state = test_encoder_model.predict(input_tensor)

    target_seq = zeros((1, 1))
    target_seq[0, 0] =  Tokenizer.word_index['<start>']
    curr_word = "<start>"
    decoded_sentence = ''
    i = 0
    while curr_word != "<end>" and i < (max_length_out - 1):
        
        output_tokens, h = test_decoder_model.predict([target_seq, state, enc_out])

        curr_token = argmax(output_tokens[0, 0])

        if (curr_token == 0):
          break;

        curr_word = index_to_word[curr_token]

        decoded_sentence += ' ' + curr_word
        target_seq[0, 0] = curr_token
        state = h
        i += 1

    return decoded_sentence[:-6]
    
###########################################################################################################################

input_text = st.text_area('', height=70) 
st.write('')

predict_button = st.button('Auto Complete')

if predict_button:
    predicted_text = decode_sequence_attention(tokenize_text(input_text))
    predicted_text = input_text + ' ' + predicted_text
    predicted_text = re.sub(r'[" "]+', " ", predicted_text)
    st.info(predicted_text)

Overwriting GmailSmartCompose.py


In [ ]:
!streamlit run GmailSmartCompose.py & npx localtunnel --port 8501

2022-02-17 12:53:48.398 INFO    numexpr.utils: NumExpr defaulting to 2 threads.
RuntimeError: module compiled against API version 0xe but this version of numpy is 0xd

  You can now view your Streamlit app in your browser.

  Network URL: http://172.28.0.2:8501
  External URL: http://35.222.154.38:8501

npx: installed 22 in 5.059s
your url is: https://itchy-badger-96.loca.lt
